In [1]:
%matplotlib inline
import numpy as np
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(precision=7)
import matplotlib.pyplot as plt
import math 
import pandas as pd
from numpy import linalg as LA
from sympy import * 

In [2]:
# Scale function
def scale(vec_1):
    
    # Test
    v1_scale = np.square(np.linalg.norm(vec_1))
#     print(v1_scale)

    v1 = np.multiply(1/np.sqrt(v1_scale + 1e-15),vec_1)
#     print(v1)
    
    return list(v1)
    
    
def QR(span):
    
    # Setup arrays
    # Original Values
    vector = []
    # Normalized Values
    vec_norm = []

    # Size of array
    row,col = span.shape
    
    # First Column
    v1 = span[:,0]
    vec_1 = v1
    vector.append(vec_1)

    v1 = scale(vec_1)
    vec_norm.append(v1)
    
    # Subsequent Columns

    start = 1
    end = col

    for j in range(start,end):

        # Column
        projection = span[:,j]
        v = span[:,j]

        #Orthonormal vector
        for i in range(len(vec_norm)):
            #projection
            # dot
            proj_dot = np.dot(v,vec_norm[i])/(np.dot(vec_norm[i],vec_norm[i]) + 1e-15)
            proj = np.multiply(proj_dot,vec_norm[i])
            projection = projection - proj

        v_norm = scale(projection)

        vec_norm.append(v_norm)
        
    # Calculate R
    Q = vec_norm
    Q = np.array(Q)
    R = np.dot(Q,span)
    R = np.array(R)

    
    return Q.T,R

def eigenspace(span):
    
    eig_vec = np.eye(span.shape[0])
    X = span.copy()

    for _ in range(100):
        Q,R = QR(X)
        eig_vec = np.dot(eig_vec,Q)
        X = np.dot(R,Q)
        eig_val = np.diag(X)
        
    return eig_val,eig_vec, X


In [3]:
def SVD(A):
    
    # Copy of array
    B = A.copy()
    
    # Size of array
    row,col = A.shape
    
    A1 = A.dot(A.T)
    print(A1)
    A2 = np.dot(A.T,A)
    print(A2)
    
    if row < col:
        # Eigenvalues
        eig_val_A1,eig_vec_A1,x_A1 = eigenspace(A1)
        
        # U: left Singular Array
        U = eig_vec_A1

        # Sigma: Middle Singular Array
        # Initialize Sigma array
        Sigma = np.zeros((row,col))
        # Initialize Eigenvalues array
        Eig = eig_val_A1*np.eye(len(eig_val_A1))

        # Create Sigma
        if row < col:
            sig = row
        elif row > col:
            sig = col
        Sigma[0:sig,0:sig] = Eig[0:sig,0:sig]
        Sigma = np.sqrt(Sigma)

        # Solve for V: Right Singular Array
        # Initialize V
        V = np.zeros((col,col))

        for i in range(len(U)):

            u1 = A.T.dot(U[:,i])
            # Normalize
            u1 = u1/(np.linalg.norm(u1) + 1e-20)

            # Update V array
            V[i,:] = u1

        # Determine V
        V = A.T.dot(U).dot(Sigma)

        for i in range(row):
            V[:,i] = V[:,i]/(np.linalg.norm(V[:,i]) + 1e-20)
            
    else:
        # Eigenvalues
        eig_val_A1,eig_vec_A1,x_A1 = eigenspace(A1)
        eig_val_A2,eig_vec_A2,x_A2 = eigenspace(A2)
        
        # Right Singular Vector
        V = eig_vec_A2
        
        # Sigma
        # Initialize Sigma array
        Sigma = np.zeros((row,col))
#         print(Sigma)
        # Initialize Eigenvalues array
        Eig = eig_val_A2*np.eye(len(eig_val_A2))
#         print(Eig)

        # Create Sigma
        if row < col:
            sig = row
        else:
            sig = col
        Sigma[0:sig,0:sig] = Eig[0:sig,0:sig]
        Sigma = np.sqrt(Sigma)
#         print(Sigma)
        
        # Solve for U
        # Initialize U
        # U = np.eye((row))
        U = np.zeros((row,row))
#         print(U)

        eig = eig_val_A2
        for i in range(len(eig)):    

            u1 = (1/(np.sqrt(eig[i]) + 1e-20))*A.dot(V[:,i])
#             print(u1)

            u1 = u1/(np.linalg.norm(u1) + 1e-20)
#             print(u1)

            U[:,i] = u1
        
        
        
    return U,Sigma,V
    


In [4]:
# A = np.array([[1,1],[1,1],[1,-1]])
# A = np.array([[2,4],[1,3],[0,0],[0,0]])
# A = np.array([[-5,1,-2,5],[0,-2,4,8]])

row = 5
col = 6
A = np.random.randint(-10,10,size=(row,col))

row,col = A.shape

print(A)

B = A.copy()

[[ -4  -1   6  -5   8   1]
 [  1   6  -2  -4   9   1]
 [-10   6   0   1  -6  -6]
 [  5   1  -5  -4   7  -8]
 [ -1  -2  -2   2  -6   6]]


In [5]:
U,Sigma,V = SVD(B)
print(U)
print(Sigma)
print(V)

[[143  71 -25  17 -58]
 [ 71 139 -38  92 -65]
 [-25 -38 209 -42   0]
 [ 17  92 -42 180 -95]
 [-58 -65   0 -95  85]]
[[ 143  -43  -49  -16   78   11]
 [ -43   78  -19  -21   29  -51]
 [ -49  -19   69   -6    7   32]
 [ -16  -21   -6   62 -122   29]
 [  78   29    7 -122  266  -39]
 [  11  -51   32   29  -39  138]]
[[ 0.3545321 -0.1117937 -0.7941608  0.3217632  0.3571922]
 [ 0.5205166 -0.129705  -0.1348108 -0.8253395 -0.1134897]
 [-0.3426768 -0.9269834 -0.0016385 -0.0873655  0.1250554]
 [ 0.57624   -0.1696637  0.5920127  0.2263777  0.4872737]
 [-0.3922875  0.2873929 -0.0255523 -0.3954789  0.778755 ]]
[[19.0581318  0.         0.         0.         0.         0.       ]
 [ 0.        13.8917538  0.         0.         0.         0.       ]
 [ 0.         0.        11.8516632  0.         0.         0.       ]
 [ 0.         0.         0.         7.13565    0.         0.       ]
 [ 0.         0.         0.         0.         2.9029929  0.       ]]
[[ 0.3044708  0.6083892  0.5099565  0.040449  -0

In [6]:
print(B)
A = U.dot(Sigma).dot(V.T)
print(A)

# Check for identity matrices
U_test = U.T.dot(U)
print(U_test)

[[ -4  -1   6  -5   8   1]
 [  1   6  -2  -4   9   1]
 [-10   6   0   1  -6  -6]
 [  5   1  -5  -4   7  -8]
 [ -1  -2  -2   2  -6   6]]
[[-4.0000000e+00 -1.0000000e+00  6.0000000e+00 -5.0000000e+00  8.0000000e+00  1.0000000e+00]
 [ 1.0000000e+00  6.0000000e+00 -2.0000000e+00 -4.0000000e+00  9.0000000e+00  1.0000000e+00]
 [-1.0000000e+01  6.0000000e+00 -1.6405541e-15  1.0000000e+00 -6.0000000e+00 -6.0000000e+00]
 [ 5.0000000e+00  1.0000000e+00 -5.0000000e+00 -4.0000000e+00  7.0000000e+00 -8.0000000e+00]
 [-1.0000000e+00 -2.0000000e+00 -2.0000000e+00  2.0000000e+00 -6.0000000e+00  6.0000000e+00]]
[[ 1.0000000e+00 -2.1510810e-17  2.2605219e-15 -8.4065988e-16 -3.2951925e-15]
 [-2.1510810e-17  1.0000000e+00 -7.9766980e-16 -9.8858310e-16  7.1444382e-15]
 [ 2.2605219e-15 -7.9766980e-16  1.0000000e+00  1.0280017e-15  8.3077813e-15]
 [-8.4065988e-16 -9.8858310e-16  1.0280017e-15  1.0000000e+00 -1.7270258e-14]
 [-3.2951925e-15  7.1444382e-15  8.3077813e-15 -1.7270258e-14  1.0000000e+00]]
